In [1]:
%matplotlib notebook

from matplotlib import pylab
from astropy.coordinates import SkyCoord
import integralclient as ic

import time
import numpy as np

from astropy.table import Table, vstack
from astropy.io import fits
import glob


In [2]:
RA=40.669879
Dec=-0.013289
tstart_rel_mdays=10
tstop_rel_days=10
t0_utc="2018-12-01T02:38:00"
cons=1
nrt=1

In [3]:
source_coord = SkyCoord(RA, Dec, unit = "deg")

In [4]:
#arc_root_prefix="/mnt/sshfs/isdc/"
arc_root_prefix="/"

In [5]:

tables=[]

if cons == 1:
    tables.append(Table.read(fits.open(arc_root_prefix+"/isdc/arc/rev_3/idx/scw/GNRL-SCWG-GRP-IDX.fits")))

if nrt == 1:
    tables.append(Table.read(fits.open(sorted(glob.glob(arc_root_prefix+"/isdc/pvphase/nrt/ops/idx/scw/GNRL-SCWG-GRP-IDX_*"))[-1])[1]))
    
print(len(tables))
    
scwd = vstack(tables)

FileNotFoundError: [Errno 2] No such file or directory: '//isdc/arc/rev_3/idx/scw/GNRL-SCWG-GRP-IDX.fits'

In [ ]:
now_ijd = float(ic.converttime("UTC",time.strftime("%Y-%m-%dT%H:%M:%S"),"IJD"))
t0_ijd =  float(ic.converttime("UTC",t0_utc,"IJD"))

tstart_ijd = t0_ijd - tstart_rel_mdays
tstop_ijd = t0_ijd + tstop_rel_days

now_ijd, t0_ijd, tstart_ijd, tstop_ijd

In [ ]:
print(scwd['TSTART'].min(), scwd['TSTOP'].max(),tstart_ijd)

scwd=scwd[(scwd['TSTART'] > tstart_ijd) & (scwd['TSTART'] < tstop_ijd)]


In [ ]:
m_point = np.array(list(map(str.strip,scwd['SW_TYPE']))) == "POINTING"
scwd=scwd[m_point]

scx=SkyCoord(scwd['RA_SCX'],scwd['DEC_SCX'],unit="deg")

offaxis=  scx.separation(source_coord) 

In [ ]:
pylab.figure(figsize=(8,6))
pylab.step(
    (scwd['TSTART']-t0_ijd)*24,
    offaxis.deg,
    #width=scwd['TELAPSE']/24./3600*24,
    
)
#tight_layout()
pylab.grid()
#pylab.xlim(-5*24,3*24)
pylab.axhspan(0,10,alpha=0.2,color="red")
pylab.axhspan(10,15,alpha=0.2,color="green")
pylab.axhspan(15,20,alpha=0.2,color="blue")
pylab.ylabel("off-axis angle, deg")
#ylim([0,50])
pylab.xlabel("hours since %.15lg (%s)"%(t0_ijd, t0_utc))

mon=(scx.separation(source_coord).degree<10) #& (scwd['TSTART']>frb_t0_ijd-5.)


print(sum(mon),sum(scwd['TELAPSE'][mon]))

pylab.savefig("ohistory.png")

In [ ]:
pylab.figure(figsize=(8,6))
pylab.hist(offaxis.deg)
pylab.grid()
pylab.xlabel("off-axis angle, deg")
#ylim([0,50])

mon=(scx.separation(source_coord).degree<10) #& (scwd['TSTART']>frb_t0_ijd-5.)


print(sum(mon),sum(scwd['TELAPSE'][mon]))

pylab.savefig("ohist.png")

In [ ]:
history_png="ohistory.png" # smth
hist_png="ohist.png" # smth else